In [1]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt 
import seaborn as sns 
import sklearn as sk 

from matplotlib import style
style.use('dark_background')

In [2]:
train_data = pd.read_csv('train_data.csv')
train_data = train_data.drop(['ID','HealthServiceArea'], axis = 1)
train_data.head()

,Gender,Race,TypeOfAdmission,CCSProcedureCode,APRSeverityOfIllnessCode,PaymentTypology,BirthWeight,EmergencyDepartmentIndicator,AverageCostInCounty,AverageChargesInCounty,AverageCostInFacility,AverageChargesInFacility,AverageIncomeInZipCode,LengthOfStay
0,F,Other Race,Newborn,228,1,Medicaid,3700,N,2611,9227,1751,8951,45,1
1,M,Black/African American,Newborn,228,1,Medicaid,2900,N,3242,8966,3338,6409,34,1
2,M,Other Race,Newborn,220,1,Private Health Insurance,3200,N,3155,11381,4980,9323,45,1
3,F,Other Race,Newborn,0,1,Private Health Insurance,3300,N,3155,11381,5826,15680,59,1
4,F,Other Race,Newborn,228,1,Medicaid,2600,N,2611,9227,6000,14344,59,1


In [3]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59966 entries, 0 to 59965
Data columns (total 14 columns):
 #   Column                        Non-Null Count  Dtype 
---  ------                        --------------  ----- 
 0   Gender                        59966 non-null  object
 1   Race                          59966 non-null  object
 2   TypeOfAdmission               59966 non-null  object
 3   CCSProcedureCode              59966 non-null  int64 
 4   APRSeverityOfIllnessCode      59966 non-null  int64 
 5   PaymentTypology               59966 non-null  object
 6   BirthWeight                   59966 non-null  int64 
 7   EmergencyDepartmentIndicator  59966 non-null  object
 8   AverageCostInCounty           59966 non-null  int64 
 9   AverageChargesInCounty        59966 non-null  int64 
 10  AverageCostInFacility         59966 non-null  int64 
 11  AverageChargesInFacility      59966 non-null  int64 
 12  AverageIncomeInZipCode        59966 non-null  int64 
 13  LengthOfStay    

In [4]:
train_data.describe()

,CCSProcedureCode,APRSeverityOfIllnessCode,BirthWeight,AverageCostInCounty,AverageChargesInCounty,AverageCostInFacility,AverageChargesInFacility,AverageIncomeInZipCode,LengthOfStay
count,59966.000000,59966.000000,59966.000000,59966.000000,59966.000000,59966.000000,59966.000000,59966.000000,59966.000000
mean,155.404229,1.254594,3336.298903,2372.806690,7979.126922,2396.414318,7958.472668,59.811143,2.538405
std,89.541978,0.546207,446.244475,639.755096,3220.291347,1248.501189,3859.294711,21.470170,1.171246
min,-1.000000,1.000000,2500.000000,712.000000,1243.000000,457.000000,1120.000000,28.000000,1.000000
25%,115.000000,1.000000,3000.000000,2041.000000,4620.000000,1551.000000,4438.000000,45.000000,2.000000
50%,220.000000,1.000000,3300.000000,2533.000000,9227.000000,1967.000000,7945.000000,55.000000,2.000000
75%,228.000000,1.000000,3600.000000,2785.000000,10644.000000,2895.000000,11619.000000,74.000000,3.000000
max,231.000000,4.000000,7500.000000,3242.000000,11381.000000,8114.000000,18466.000000,115.000000,10.000000


In [5]:
X = train_data.drop('LengthOfStay', axis = 1)
Y = train_data['LengthOfStay'].apply(lambda x: 0 if int(x) <= 3 else 1)

In [6]:
Y.value_counts()

0    49895
1    10071
Name: LengthOfStay, dtype: int64

In [7]:
for col in X.columns:
    print(X[col].value_counts())
    print('---------------------------')

M    30978
F    28987
U        1
Name: Gender, dtype: int64
---------------------------
White                     32943
Other Race                18314
Black/African American     8183
Multi-racial                526
Name: Race, dtype: int64
---------------------------
Newborn      58741
Emergency      659
Urgent         412
Elective       154
Name: TypeOfAdmission, dtype: int64
---------------------------
 228    19886
 115    13628
 0      11189
 220    10773
 231     2981
-1        769
 216      740
Name: CCSProcedureCode, dtype: int64
---------------------------
1    47953
2     8760
3     3252
4        1
Name: APRSeverityOfIllnessCode, dtype: int64
---------------------------
Medicaid                     28723
Private Health Insurance     15608
Blue Cross/Blue Shield       12073
Self-Pay                      1984
Federal/State/Local/VA         849
Managed Care, Unspecified      545
Miscellaneous/Other            118
Medicare                        44
Unknown                        

## Dummy Encode

In [8]:
# Select columns for one-hot encoding
# CSS Procedure Code not sure
encode_col = ['Gender','Race','TypeOfAdmission','CCSProcedureCode','PaymentTypology','EmergencyDepartmentIndicator']
X_copy = X.copy()


for col in encode_col:
    # create new dataframe for each type of encode_column
    dummiesDF = pd.get_dummies(X_copy[col])
    # merge the new dataframe into the orginal
    X_copy = pd.concat([X_copy, dummiesDF], axis='columns')
    # Drop the original column after encoding
#     if col == 'Gender':
#         X_copy = X_copy.drop([col,'U'], axis = 1)
#     else:
    X_copy = X_copy.drop(col, axis = 1)

X_copy.columns


Index([ 'APRSeverityOfIllnessCode',               'BirthWeight',
             'AverageCostInCounty',    'AverageChargesInCounty',
           'AverageCostInFacility',  'AverageChargesInFacility',
          'AverageIncomeInZipCode',                         'F',
                               'M',                         'U',
          'Black/African American',              'Multi-racial',
                      'Other Race',                     'White',
                        'Elective',                 'Emergency',
                         'Newborn',                    'Urgent',
                                -1,                           0,
                               115,                         216,
                               220,                         228,
                               231,    'Blue Cross/Blue Shield',
          'Federal/State/Local/VA', 'Managed Care, Unspecified',
                        'Medicaid',                  'Medicare',
             'Miscellaneo

## Test the data in training dataset

In [9]:
from sklearn.model_selection import train_test_split

train_X, test_X, train_Y, test_Y = train_test_split(X_copy,Y,test_size = 0.3, stratify = Y)

print(train_X.shape)
print(test_X.shape)
print(train_Y.shape)
print(test_Y.shape)

(41976, 36)
(17990, 36)
(41976,)
(17990,)


In [10]:
# Reset index after spliting

def df_reset_index(x):
    return x.reset_index().drop('index', axis = 1)

train_X = df_reset_index(train_X)
test_X = df_reset_index(test_X)
train_Y = df_reset_index(train_Y)
test_Y = df_reset_index(test_Y)

In [11]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(train_X)
train_X_scale = scaler.transform(train_X)
test_X_scale = scaler.transform(test_X)
train_Y_ravel = train_Y.values.ravel()

In [12]:
from sklearn.linear_model import LogisticRegression

log_reg = LogisticRegression(random_state = 0, max_iter = 1000)
log_reg.fit(train_X_scale, train_Y_ravel)
print('Intercept:', log_reg.intercept_)
print('Coefficient/Theta', log_reg.coef_)

Intercept: [-1.78694187]
Coefficient/Theta [[ 0.54004582 -0.04993377  0.00247297  0.00105276  0.14880485  0.02725258
   0.26222462 -0.00176042  0.00176042  0.          0.08234723 -0.0289812
  -0.00428196 -0.04728965 -0.14196499 -0.19732407  0.19460002 -0.00171847
   0.07103552 -0.12743938  0.06426986  0.01493929 -0.05756913 -0.04053465
   0.24773956  0.02551469 -0.01571466  0.02561137 -0.04843849 -0.01746438
   0.02099648  0.0495193  -0.05104636  0.02114076 -0.01101062  0.01101062]]


In [13]:
from sklearn.metrics import accuracy_score, mean_squared_error

log_pred = log_reg.predict(test_X_scale)
print('Accuracy:',accuracy_score(test_Y,log_pred))
print('Mean squared error ', mean_squared_error(test_Y, log_pred))

Accuracy: 0.8392996108949417
Mean squared error  0.16070038910505838


In [14]:
unique, counts = np.unique(log_pred, return_counts=True)
unique_Y, counts_Y = np.unique(test_Y, return_counts=True)
print('Prediction:',dict(zip(unique, counts)))
print('Test_Y:',dict(zip(unique_Y, counts_Y)))

Prediction: {0: 17272, 1: 718}
Test_Y: {0: 14969, 1: 3021}


## Lasso

In [15]:
from sklearn.linear_model import Lasso 

# A list to hold different values of alpha
Log_Reg_Para = [0.01,0.05,0.1,0.25,0.5,0.75,1]

for para in Log_Reg_Para:
    # Create the logistic regression object
    lasso = Lasso(alpha = para, max_iter=1000)
    
    # Fit the model
    lasso.fit(train_X_scale, train_Y_ravel)
    
    # Predict
    preds = lasso.predict(test_X_scale)
    
    # MSE & Accuracy
    mse = mean_squared_error(test_Y, preds)
    print("Alpha MSE",para,":",mse)
    print('---------------------------')

Alpha MSE 0.01 : 0.12595430617147
---------------------------
Alpha MSE 0.05 : 0.13176542723754156
---------------------------
Alpha MSE 0.1 : 0.13909573365442382
---------------------------
Alpha MSE 0.25 : 0.13972727491774828
---------------------------
Alpha MSE 0.5 : 0.13972727491774828
---------------------------
Alpha MSE 0.75 : 0.13972727491774828
---------------------------
Alpha MSE 1 : 0.13972727491774828
---------------------------


In [16]:
lasso = Lasso(alpha = 0.01, max_iter=1000)
# Fit the model
lasso.fit(train_X_scale, train_Y_ravel)
# Predict
preds = lasso.predict(test_X_scale)
preds

array([0.09649393, 0.11607457, 0.18764098, ..., 0.17087706, 0.15336643,
       0.11101855])

## Apply model with test dataset

In [44]:
test_data = pd.read_csv('test_data.csv')
submission = pd.DataFrame(columns=['ID','LengthOfStay'])
submission['ID'] = test_data['ID']
test_data = test_data.drop(['ID','HealthServiceArea'], axis = 1)
test_data.head()

,Gender,Race,TypeOfAdmission,CCSProcedureCode,APRSeverityOfIllnessCode,PaymentTypology,BirthWeight,EmergencyDepartmentIndicator,AverageCostInCounty,AverageChargesInCounty,AverageCostInFacility,AverageChargesInFacility,AverageIncomeInZipCode
0,F,Other Race,Newborn,216,3,Medicaid,4900,N,3242,8966,2448,13196,34
1,F,White,Newborn,220,1,Medicaid,3100,N,2611,9227,1751,8951,45
2,M,Multi-racial,Newborn,115,1,Medicaid,3300,N,3155,11381,1052,11314,74
3,M,Other Race,Newborn,228,1,Medicaid,3300,N,2611,9227,3790,6250,45
4,F,Other Race,Newborn,220,2,Medicaid,3800,N,3155,11381,3032,10135,59


In [45]:
test_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 69177 entries, 0 to 69176
Data columns (total 13 columns):
 #   Column                        Non-Null Count  Dtype 
---  ------                        --------------  ----- 
 0   Gender                        69177 non-null  object
 1   Race                          69177 non-null  object
 2   TypeOfAdmission               69177 non-null  object
 3   CCSProcedureCode              69177 non-null  int64 
 4   APRSeverityOfIllnessCode      69177 non-null  int64 
 5   PaymentTypology               69177 non-null  object
 6   BirthWeight                   69177 non-null  int64 
 7   EmergencyDepartmentIndicator  69177 non-null  object
 8   AverageCostInCounty           69177 non-null  int64 
 9   AverageChargesInCounty        69177 non-null  int64 
 10  AverageCostInFacility         69177 non-null  int64 
 11  AverageChargesInFacility      69177 non-null  int64 
 12  AverageIncomeInZipCode        69177 non-null  int64 
dtypes: int64(8), obj

In [46]:
for col in test_data.columns:
    print(test_data[col].value_counts())
    print('---------------------------')

M    35471
F    33704
U        2
Name: Gender, dtype: int64
---------------------------
White                     30432
Other Race                27043
Black/African American    10780
Multi-racial                922
Name: Race, dtype: int64
---------------------------
Newborn      68125
Emergency      495
Urgent         362
Elective       194
Trauma           1
Name: TypeOfAdmission, dtype: int64
---------------------------
 228    24212
 0      15120
 220    13108
 115    12819
 231     2535
 216      764
-1        619
Name: CCSProcedureCode, dtype: int64
---------------------------
1    55243
2    10935
3     2998
4        1
Name: APRSeverityOfIllnessCode, dtype: int64
---------------------------
Medicaid                     37037
Private Health Insurance     14871
Blue Cross/Blue Shield       12795
Self-Pay                      2508
Managed Care, Unspecified     1539
Federal/State/Local/VA         320
Medicare                        43
Miscellaneous/Other             27
Unknown     

In [47]:
for col in encode_col:
    # create new dataframe for each type of encode_column
    dummiesDF = pd.get_dummies(test_data[col])
    # merge the new dataframe into the orginal
    test_data = pd.concat([test_data, dummiesDF], axis='columns')
    # Drop the original column after encoding
    test_data = test_data.drop(col, axis = 1)

test_data.columns

Index([ 'APRSeverityOfIllnessCode',               'BirthWeight',
             'AverageCostInCounty',    'AverageChargesInCounty',
           'AverageCostInFacility',  'AverageChargesInFacility',
          'AverageIncomeInZipCode',                         'F',
                               'M',                         'U',
          'Black/African American',              'Multi-racial',
                      'Other Race',                     'White',
                        'Elective',                 'Emergency',
                         'Newborn',                    'Trauma',
                          'Urgent',                          -1,
                                 0,                         115,
                               216,                         220,
                               228,                         231,
          'Blue Cross/Blue Shield', 'Department of Corrections',
          'Federal/State/Local/VA', 'Managed Care, Unspecified',
                        '

In [48]:
test_data = test_data.drop(['Trauma','Department of Corrections'], axis = 1)

In [49]:
scaler_test = StandardScaler()
scaler_test.fit(test_data)
test_data_scale = scaler_test.transform(test_data)

In [50]:
prediction = log_reg.predict(test_data_scale)
prediction

array([0, 0, 0, ..., 0, 0, 0], dtype=int64)

In [51]:
u, c = np.unique(prediction, return_counts=True)
print('Prediction:',dict(zip(u, c)))

Prediction: {0: 66444, 1: 2733}


In [52]:
submission['LengthOfStay'] = pd.Series(prediction)
submission

,ID,LengthOfStay
0,1,0
1,2,0
2,3,0
3,4,0
4,5,0
...,...,...
69172,69173,0
69173,69174,0
69174,69175,0
69175,69176,0


In [55]:
submission.to_csv('test1.csv',index=False)